In [128]:
import nltk, re
from nltk.corpus import gutenberg
import matplotlib.pyplot as plt
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import words, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


%matplotlib inline

In [129]:
#A function to work out English plurals. 
#Takes a singular noun and generates a plural form, though it is not always correct.
 	
def plural(word):
    if word.endswith('y'):
        return word[:-1] + 'ies'
    elif word[-1] in 'sx' or word[-2:] in ['sh', 'ch']:
        return word + 'es'
    elif word.endswith('an'):
        return word[:-2] + 'en'
    else:
        return word + 's'

In [132]:
#Test the function plural() with 5 different nouns, two of which won't be in correct plural form generated by this function.

#Test subject
test = ["baby", "box", "Man", "House", "Book"]

#Test plural()
for word in test:
    print("noun:", test)
    print("Plural:", plural(word))
    print()

noun: ['baby', 'box', 'Man', 'House', 'Book']
Plural: babies

noun: ['baby', 'box', 'Man', 'House', 'Book']
Plural: boxes

noun: ['baby', 'box', 'Man', 'House', 'Book']
Plural: Men

noun: ['baby', 'box', 'Man', 'House', 'Book']
Plural: Houses

noun: ['baby', 'box', 'Man', 'House', 'Book']
Plural: Books



In [134]:
#Pick of one the texts from gutenberg corpus and display the lines 10 through 25 from the text

#Pulling gutenberg
from nltk.corpus import gutenberg

sample = gutenberg.raw('austen-emma.txt')

#Split
lines = sample.split('\n')

#lines 10 - 25
for i in range(10, 26):
    print(f"Line {i + 1}:", lines[i])

Line 11: with very little to distress or vex her.
Line 12: 
Line 13: She was the youngest of the two daughters of a most affectionate,
Line 14: indulgent father; and had, in consequence of her sister's marriage,
Line 15: been mistress of his house from a very early period.  Her mother
Line 16: had died too long ago for her to have more than an indistinct
Line 17: remembrance of her caresses; and her place had been supplied
Line 18: by an excellent woman as governess, who had fallen little short
Line 19: of a mother in affection.
Line 20: 
Line 21: Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
Line 22: less as a governess than a friend, very fond of both daughters,
Line 23: but particularly of Emma.  Between _them_ it was more the intimacy
Line 24: of sisters.  Even before Miss Taylor had ceased to hold the nominal
Line 25: office of governess, the mildness of her temper had hardly allowed
Line 26: her to impose any restraint; and the shadow of authority being


In [136]:
#Obtain the list of words for the text you picked and display the last 10 words

from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer

# extract lines 11–26
emma_lines = gutenberg.raw('austen-emma.txt').split('\n')
section_text = '\n'.join(emma_lines[10:26])

# Had to use RegexpTokenizer because I was getting Punkt errors
tokenizer = RegexpTokenizer(r"\w+")
section_tokens = tokenizer.tokenize(section_text)

# display the last 10 tokens
print("Last 10 tokens of lines 11–26:", section_tokens[-10:])


Last 10 tokens of lines 11–26: ['to', 'impose', 'any', 'restraint', 'and', 'the', 'shadow', 'of', 'authority', 'being']


In [138]:
## A function that finds words in the input text that are not in the standard English vocabulary.
# It computes the vocabulary of a text, then removes all items that occur in an existing wordlist, 
# leaving just the uncommon or mis-spelt words.

def unusual_words(text):
    text_vocab = set(w.lower() for w in text if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.words.words()) 
    unusual = text_vocab - english_vocab
    return sorted(unusual)


#example use
unusual_words(['hello','NLP','AI'])

['nlp']

In [140]:
##Use the function above to find unusual/mis-spelt words in your text that you picked from Gutenberg corpus earlier. 

words = unusual_words(section_tokens)

print("Unusual/mis-spelt words:")
print(words)

Unusual/mis-spelt words:
['allowed', 'caresses', 'ceased', 'daughters', 'died', 'fond', 'sisters', 'supplied', 'years', 'youngest']


In [150]:
#Modify the above function to include lemmatization applied to the text 

#Using lemmatizer from Wordnet
lemmatizer = WordNetLemmatizer()

def lem_unusual_words(text):
    text_vocab = set(w.lower() for w in text if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.words.words()) 
    
    #try lemmatization for our text
    lemmatized_vocab = set(lemmatizer.lemmatize(w) for w in text_vocab)
    
    #use lemmatized words from our text to find unusual words
    unusual = lemmatized_vocab - english_vocab
    
    return sorted(unusual)


In [152]:
##Use the modified function for your text you picked from the Gutenberg corpus

# Use the lemmatized unusual words function on section_tokens (Emma lines 11–26)
lemmatized_unusual = lem_unusual_words(section_tokens)

# Print the result
print("Lemmatized unusual/mis-spelled words from the Emma section:")
print(lemmatized_unusual)


Lemmatized unusual/mis-spelled words from the Emma section:
['allowed', 'ceased', 'died', 'fond', 'supplied', 'youngest']


In [50]:
#Comment on differences in output you observe. Explain why. 
#
#in the lummatized form it removed some words because they were turned singular

In [51]:
#A function to compute what fraction of words in a text are not in the stopwords list:
def content_fraction(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return len(content) / len(text)

#example use
content_fraction("This is an example sentence with stopwords." )


0.5813953488372093

In [82]:
#Here we calculate bag of words representation. We use CountVectorizer from sklearn.
from sklearn.feature_extraction.text import CountVectorizer

#Our text is preprocessed to remove period.
documents = ["Dog bites man.", 
             "My dog loves to eat his food.", 
             "The dog eats a bone.", 
             "The dog scratched the cat", 
             "Man eats food.", 
             "Dog eats grass.", ] 
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

#look at the documents list
print("Our corpus: ", processed_docs)

count_vect = CountVectorizer()
#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

#Example use: see the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'my dog loves to eat his food.: ",bow_rep[1].toarray())



Our corpus:  ['dog bites man', 'my dog loves to eat his food', 'the dog eats a bone', 'the dog scratched the cat', 'man eats food', 'dog eats grass']
Our vocabulary:  {'dog': 3, 'bites': 0, 'man': 10, 'my': 11, 'loves': 9, 'to': 14, 'eat': 4, 'his': 8, 'food': 6, 'the': 13, 'eats': 5, 'bone': 1, 'scratched': 12, 'cat': 2, 'grass': 7}
BoW representation for 'dog bites man':  [[1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]]
BoW representation for 'my dog loves to eat his food.:  [[0 0 0 1 1 0 1 0 1 1 0 1 0 0 1]]


In [80]:

#Use the function, content_fraction, to find the fraction of words in your text 
#(the one you picked earlier from the Gutenberg corpus)
#are not in the stopwords list and comment on the result

import nltk
nltk.download('stopwords')

#Uses above function
fraction = content_fraction(section_tokens)

print("Fraction:", round(fraction, 2))


Fraction: 0.44


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\usaas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#from 11-24 my output is .44 showing 44 percent is meaningful

In [54]:
#Here we calculate bag of words representation. We use CountVectorizer from sklearn.
from sklearn.feature_extraction.text import CountVectorizer

#Our text is preprocessed to remove period.
documents = ["Dog bites man.", 
             "My dog loves to eat his food.", 
             "The dog eats a bone.", 
             "The dog scratched the cat", 
             "Man eats food.", 
             "Dog eats grass.", ] 
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

#look at the documents list
print("Our corpus: ", processed_docs)

count_vect = CountVectorizer()
#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

#Example use: see the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'my dog loves to eat his food.: ",bow_rep[1].toarray())



In [100]:
## Get the BoW representation using this vocabulary, for a new text of your choice

text = ["My dog loves to eat my food."]

#Transform using vectorize
bow = count_vect.transform([text[0].lower().replace(".","")])

#Prints BoW vector
print("BoW:", bow.toarray())

BoW: [[0 0 0 1 1 0 1 0 0 1 0 2 0 0 1]]


******************************************************
## TDIDF

In BoW, all the words in the text are treated equally important. There is no notion of some words in the document being more important than others. TF-IDF addresses this issue. It aims to quantify the importance of a given word relative to other words in the document and in the corpus. It was commonly used representation scheme for information retrieval systems, for extracting relevant documents from a corpus for given text query.

The next cell shows a simple example of how to get the TF-IDF representation of a document using sklearn's TfidfVectorizer.

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(processed_docs)

#IDF for all words in the vocabulary
print("IDF for all words in the vocabulary",tfidf.idf_)
print("-"*10)
#All words in the vocabulary.
print("All words in the vocabulary",tfidf.get_feature_names_out())
print("-"*10)

#TFIDF representation for all documents in our corpus 
print("TFIDF representation for all documents in our corpus\n",bow_rep_tfidf.toarray()) 
print("-"*10)





IDF for all words in the vocabulary [2.25276297 2.25276297 2.25276297 1.15415068 2.25276297 1.55961579
 1.84729786 2.25276297 2.25276297 2.25276297 1.84729786 2.25276297
 2.25276297 1.84729786 2.25276297]
----------
All words in the vocabulary ['bites' 'bone' 'cat' 'dog' 'eat' 'eats' 'food' 'grass' 'his' 'loves'
 'man' 'my' 'scratched' 'the' 'to']
----------
TFIDF representation for all documents in our corpus
 [[0.71890333 0.         0.         0.36831339 0.         0.
  0.         0.         0.         0.         0.58951102 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.21030046 0.41048115 0.
  0.33660042 0.         0.41048115 0.41048115 0.         0.41048115
  0.         0.         0.41048115]
 [0.         0.64359624 0.         0.32973156 0.         0.44556967
  0.         0.         0.         0.         0.         0.
  0.         0.52775813 0.        ]
 [0.         0.         0.44936797 0.23022322 0.         0.
  0.         0.         0.         0.    

In [94]:
#Get the TFIDF representation using this vocabulary, for the text of your choice from earlier

#New sentence
text = ["My dog loves to eat my sisters food."]

#TFIDF vectorizer
tfidf = tfidf.transform([text[0].lower().replace(".", "")])

#TFIDF vector
print("TFIDF representation:\n", tfidf.toarray())


TFIDF representation:
 [[0.         0.         0.         0.18187635 0.3550007  0.
  0.29110565 0.         0.         0.3550007  0.         0.71000141
  0.         0.         0.3550007 ]]


**************************************************************

### Precision, Recall, F1

A Named Entity Recognition (NER) system is evaluated on a dataset with three entity types: PERSON, ORG, and LOC. The confusion matrix below shows how the system performed:
Calculate the Precision, Recall, and F1 Score for the PERSON class.

|                    | Predicted: PERSON | Predicted: ORG | Predicted: LOC |
| ------------------ | ----------------- | -------------- | -------------- |
| **Actual: PERSON** | 45                | 5              | 3              |
| **Actual: ORG**    | 4                 | 45             | 7              |
| **Actual: LOC**    | 3                 | 3              | 48             |


In [98]:
#Calculate the Precision, Recall, and F1 Score for the PERSON class.
#Show your work

#TP = 45, FP = 7, Gn = 8

#Precision = TP / TP + FP so 45 / 45+7 = 0.865
#Recall = TP / TP + FN so 45 / 45+8 = 0.849
#F1 = 2 x precision x recall / precision + recall = 2 x 0.865 x 0.849 / 0.865 + 0.849 = 0.857


TP = 45
FP = 4 + 3
FN = 5 + 3

# Precision, Recall, F1 calculations
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

#Coded calculations
print("Precision for PERSON:", round(precision, 3))
print("Recall for PERSON:", round(recall, 3))
print("F1 Score for PERSON:", round(f1_score, 3))


Precision for PERSON: 0.865
Recall for PERSON: 0.849
F1 Score for PERSON: 0.857
